In [77]:
import pandas as pd
train=pd.read_csv("train_preprocess.csv",index_col=0)
test=pd.read_csv("test_preprocess.csv",index_col=0)
test=test.drop(axis=1,labels=['index'])
y=pd.read_csv("train.csv")
submit=pd.read_csv("sample_submission.csv")
print(y.shape)
train.head()

(26457, 20)


,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,0,0,0,0,0,0,3,1,2,3,1,0,0,0,0,2.0,0
1,0,0,1,1,0,0,2,0,1,2,1,0,0,1,1,3.0,0
2,1,1,1,0,1,4,3,1,1,6,1,0,1,0,2,2.0,1
3,0,0,1,0,0,0,2,1,1,4,1,0,1,0,3,2.0,2
4,0,1,1,0,0,2,3,1,1,4,1,0,0,0,2,2.0,1


In [78]:
y=y['credit']
test=test.drop(axis=1,labels=['FLAG_MOBIL'])

In [79]:
print(train.shape)
print(test.shape)
print(y.shape)

(26457, 17)
(10000, 17)
(26457,)


In [80]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

In [81]:
x_train,x_valid,y_train,y_valid=train_test_split(train,y,test_size=0.15,stratify=y)

In [82]:
print("x_train : ",x_train.shape,"y_train :", y_train.shape,"x_valid :", x_valid.shape,"y_valid :",y_valid.shape)

x_train :  (22488, 17) y_train : (22488,) x_valid : (3969, 17) y_valid : (3969,)


#XGBoost

In [83]:
xgb_classifier=xgb.XGBClassifier()
xgb_classifier.fit(x_train,y_train)

y_train_pred = xgb_classifier.predict(x_train)
y_valid_pred= xgb_classifier.predict(x_valid)

In [84]:
rmse_train =np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_valid = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
print('RMSE train %0.3f, valid: %0.3f' %(rmse_train, rmse_valid))

RMSE train 0.848, valid: 0.849


In [85]:
xgb_classifier.fit(train,y)
y_test_pred_proba = xgb_classifier.predict_proba(test)
y_test_pred_proba

array([[0.10800172, 0.16236599, 0.7296323 ],
       [0.11977746, 0.1912658 , 0.68895674],
       [0.13324003, 0.17939436, 0.68736565],
       ...,
       [0.10689813, 0.16729015, 0.7258117 ],
       [0.09026609, 0.155263  , 0.7544709 ],
       [0.11094995, 0.3542541 , 0.53479594]], dtype=float32)

In [86]:
submit.iloc[:,1:]=y_test_pred_proba
submit.head()

,index,0,1,2
0,26457,0.108002,0.162366,0.729632
1,26458,0.119777,0.191266,0.688957
2,26459,0.133240,0.179394,0.687366
3,26460,0.113378,0.152527,0.734095
4,26461,0.123629,0.339555,0.536816


In [94]:
submit.to_csv("sample_submission_ver1_xgBoost.csv", index = False)

#CaTBoost

In [71]:
!pip install catboost

     |████████████████████████████████| 67.3MB 54kB/s 


In [88]:
from catboost import CatBoostClassifier
cat_classifier = CatBoostClassifier(iterations=1000)
cat_classifier.fit(x_train,y_train,verbose=False)

In [89]:
y_train_pred = cat_classifier.predict(x_train)
y_valid_pred= cat_classifier.predict(x_valid)

In [90]:
rmse_train =np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_valid = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
print('RMSE train %0.3f, valid: %0.3f' %(rmse_train, rmse_valid))

RMSE train 0.763, valid: 0.815


In [91]:
cat_classifier.fit(train,y,verbose=False)
y_test_pred_proba = cat_classifier.predict_proba(test)
y_test_pred_proba

array([[0.1911562 , 0.05693709, 0.75190671],
       [0.16550347, 0.09202023, 0.7424763 ],
       [0.18942676, 0.21007883, 0.60049441],
       ...,
       [0.03455537, 0.11230387, 0.85314076],
       [0.08791726, 0.27364492, 0.63843781],
       [0.05006477, 0.51776065, 0.43217458]])

In [92]:
submit.iloc[:,1:]=y_test_pred_proba
submit.head()

,index,0,1,2
0,26457,0.191156,0.056937,0.751907
1,26458,0.165503,0.092020,0.742476
2,26459,0.189427,0.210079,0.600494
3,26460,0.083291,0.096015,0.820694
4,26461,0.060560,0.284760,0.654680


In [93]:
submit.to_csv("sample_submission_ver1_CaT.csv", index = False)